In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [7]:
from tensor2tensor.utils import beam_search

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer, initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        
        def forward(x, y, reuse = False):
            batch_size = tf.shape(x)[0]
            X_seq_len = tf.count_nonzero(x, 1, dtype = tf.int32)
            Y_seq_len = tf.count_nonzero(y, 1, dtype = tf.int32)
            with tf.variable_scope('model',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(embeddings, x)
                decoder_embedded = tf.nn.embedding_lookup(embeddings, y)
                rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
                last_output, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded,
                                                  sequence_length=X_seq_len,
                                                  dtype = tf.float32)
                
            with tf.variable_scope("decoder",reuse=reuse):
                
                attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = last_output)
                rnn_cells = tf.contrib.seq2seq.AttentionWrapper(
                    cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
                    attention_mechanism = attention_mechanism,
                    attention_layer_size = size_layer)
                
                initial_state = rnn_cells.zero_state(batch_size, tf.float32).clone(cell_state=last_state)
                outputs, _ = tf.nn.dynamic_rnn(rnn_cells, decoder_embedded, 
                                               sequence_length=Y_seq_len,
                                               initial_state = initial_state,
                                               dtype = tf.float32)
                
                return tf.layers.dense(outputs,vocab_size)
            
        main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.training_logits = forward(self.X, decoder_input, reuse = False)
        
        self.training_logits = self.training_logits[:, :tf.reduce_max(self.Y_seq_len)]
        self.training_logits = pad_second_dim(self.training_logits, tf.reduce_max(self.Y_seq_len))
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        initial_ids = tf.fill([batch_size], GO)
        def symbols_to_logits(ids):
            x = tf.contrib.seq2seq.tile_batch(self.X, 1)
            logits = forward(x, ids, reuse = True)
            return logits[:, tf.shape(ids)[1]-1, :]
        
        final_ids, final_probs, _ = beam_search.beam_search(
            symbols_to_logits,
            initial_ids,
            1,
            tf.reduce_max(self.X_seq_len),
            vocab_size,
            0.0,
            eos_id = EOS)
        
        self.fast_result = final_ids

In [8]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/

In [10]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [11]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[[    1, 11384,   156, 25249, 25249, 20948, 20948, 20948, 20948,
          20948, 19153,  9072, 20427, 20427, 20427, 17640, 17640, 17745,
          17640, 17745,  8047,  8047,  8047,   298, 30431, 21464, 21464,
          21464, 25557, 25557, 10389, 10389,  7755,  7755, 13797,  8054,
           8054]],
 
        [[    1, 26786, 26786,  3769,  3769,  3769, 10479, 30326, 10479,
          10479, 28758, 28758, 28758,  5723,  5723, 28758, 26049, 23921,
          23921, 23921,  6841,  6841, 13303, 30190, 30190, 30190, 30190,
          26982, 26982, 26982, 16855, 16855, 16855,  7604,  7604, 28847,
           5231]],
 
        [[    1, 12601, 13766,  4975,  4975,  3488,  3488,  3488,  3488,
           3488, 13643, 12133, 21723, 26334,  7654,  7654,  7654,  3799,
           3799, 13357, 13357, 13357, 13357, 31248,  7655,  7655, 27641,
          25167, 25167, 25167, 25167, 31248,  1984, 23881, 24459, 24459,
          24459]],
 
        [[    1,  6828,  6828, 18659, 29627, 24190, 21607, 21

In [12]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 6.969027, training avg acc 0.117139
epoch 1, testing avg loss 6.357158, testing avg acc 0.146500


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 5.990998, training avg acc 0.158711
epoch 2, testing avg loss 5.820802, testing avg acc 0.163295


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 18, training avg loss 3.255657, training avg acc 0.363367
epoch 18, testing avg loss 6.400053, testing avg acc 0.158134


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 19, training avg loss 3.191344, training avg acc 0.373351
epoch 19, testing avg loss 6.540891, testing avg acc 0.153772


minibatch loop: 100%|██████████| 40/40 [00:06<00:00,  6.46it/s, accuracy=0.172, cost=6.69]

epoch 20, training avg loss 3.135640, training avg acc 0.381931
epoch 20, testing avg loss 6.601688, testing avg acc 0.157739


In [13]:
from tensor2tensor.utils import bleu_hook

In [14]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)[:,0,:]
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 40/40 [02:44<00:00,  4.12s/it]


In [15]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [16]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.053475615